<a href="https://colab.research.google.com/github/hyonnys/tp2/blob/main/AIB_tp2_RTMDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Setup the environment

* Mount Google Drive
* Install framework

[Official Document for MMYolo Object Detection](https://mmyolo.readthedocs.io/en/latest/get_started/15_minutes_object_detection.html#installation)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install openmim
!mim install "mmengine>=0.6.0"
!mim install "mmcv>=2.0.0rc4,<2.1.0"
!mim install "mmdet>=3.0.0,<4.0.0"

In [ ]:
!git clone https://github.com/open-mmlab/mmyolo.git
%cd mmyolo
# Install albumentations
!pip install -r requirements/albu.txt
# Install MMYOLO
!mim install -v -e .

# 1.Download the data

In [ ]:
!pwd
%cd /content

In [ ]:
# !mkdir -pv data/coco
!mkdir -pv data/coco

In [ ]:
# Download the annotations.json(COCO file)-labels
import zipfile

ANNO_ZIP = '/content/drive/MyDrive/AIB_tp2/data/annotations.zip'

# OUTPUT_DIR = '/content/data/coco'
OUTPUT_DIR = '/content/data/coco'

with zipfile.ZipFile(ANNO_ZIP, 'r') as zip_ref:
  zip_ref.extractall(OUTPUT_DIR)

print("Successfully extracted the zip file.")

In [ ]:
# Download the raw dataset
DATA_ZIP = '/content/drive/MyDrive/AIB_tp2/data/dataset.zip'

OUTPUT_DIR = '/content/data/coco'

with zipfile.ZipFile(DATA_ZIP, 'r') as zip_ref:
  zip_ref.extractall(OUTPUT_DIR)

# 2.Train a model

[Official Github page of RTMDet](https://github.com/open-mmlab/mmdetection/tree/main/configs/rtmdet)

In [ ]:
# Download the model
!pwd
%cd /content/mmyolo/
!mkdir checkpoints

!wget -c https://download.openmmlab.com/mmrazor/v1/rtmdet_distillation/kd_tiny_rtmdet_s_neck_300e_coco/kd_tiny_rtmdet_s_neck_300e_coco_20230213_104240-e1e4197c.pth\
-O checkpoints/custom_tiny_rtmdet_coco_20230824.pth

In [ ]:
# Create a config file
!cp configs/rtmdet/distillation/kd_tiny_rtmdet_s_neck_300e_coco.py configs/rtmdet/distillation/custom_tiny_rtmdet_coco.py

In [ ]:
classes=['오토바이_보행자도로 통행위반', '오토바이_안전모 미착용',\
                 '오토바이_무단횡단','오토바이_신호위반','오토바이_정지선위반',\
                 '오토바이_횡단보도 주행위반','자전거 캐리어',\
                 '자전거_보행자도로 통행위반', '자전거_안전모 미착용', '자전거_무단횡단',\
                 '자전거_신호위반', '자전거_정지선위반', '자전거_횡단보도 주행위반',\
                 '킥보드 캐리어', '킥보드_보행자도로 통행위반',\
                 '킥보드_안전모 미착용', '킥보드_무단횡단', '킥보드_신호위반',\
                 '킥보드_횡단보도 주행위반', '킥보드_동승자 탑승위반']
len(classes)

In [ ]:
# Modify a config file
config_rtmdet_tiny="""
_base_='kd_tiny_rtmdet_s_neck_300e_coco.py'

# Modify the train/val dataset
data_root='/content/data/coco/'
metainfo = {
    'classes': ('오토바이_보행자도로 통행위반', '오토바이_안전모 미착용',\
                 '오토바이_무단횡단','오토바이_신호위반','오토바이_정지선위반',\
                 '오토바이_횡단보도 주행위반','자전거 캐리어',\
                 '자전거_보행자도로 통행위반', '자전거_안전모 미착용', '자전거_무단횡단',\
                 '자전거_신호위반', '자전거_정지선위반', '자전거_횡단보도 주행위반',\
                 '킥보드 캐리어', '킥보드_보행자도로 통행위반',\
                 '킥보드_안전모 미착용', '킥보드_무단횡단', '킥보드_신호위반',\
                 '킥보드_횡단보도 주행위반', '킥보드_동승자 탑승위반'),
    'palette': [
        (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),\
        (0, 255, 255), (255, 128, 0), (128, 0, 255), (0, 255, 128), (255, 128, 128),\
        (128, 255, 128), (128, 128, 255), (128, 128, 0), (128, 0, 128), (0, 128, 128),\
        (192, 64, 0), (192, 192, 64), (64, 192, 192), (64, 64, 192), (192, 64, 192),\
        (64, 192, 64), (255, 192, 128), (128, 255, 192), (128, 192, 255)
    ]}
train_ann_file='annotations/train_annotations.json'
train_data_prefix='train/'
val_ann_file='annotations/val_annotations.json'
val_data_prefix='val/'
test_ann_file='annotations/test_annotations.json'
test_data_prefix='test/'
img_scale=(1920,1080)
val_batch_size_per_gpu = 32
val_num_workers = 10
persistent_workers = True
dataset_type = 'YOLOv5CocoDataset'
batch_shapes_cfg = dict(
    type='BatchShapePolicy',
    batch_size=val_batch_size_per_gpu,
    img_size=img_scale[0],
    size_divisor=32,
    extra_pad_ratio=0.5)

# setup the metainfo for train/val
train_dataloader=dict(
  dataset=dict(
    metainfo=metainfo
  ))
val_dataloader=dict(
  dataset=dict(
    metainfo=metainfo
  ))

# Modify test dataset
test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=_base_.backend_args),
    dict(type='YOLOv5KeepRatioResize', scale=img_scale),
    dict(
        type='LetterResize',
        scale=img_scale,
        allow_scale_up=False,
        pad_val=dict(img=114)),
    dict(type='LoadAnnotations', with_bbox=True, _scope_='mmdet'),
    dict(
        type='mmdet.PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor', 'pad_param'))
]
test_dataloader=dict(
  batch_size=val_batch_size_per_gpu,
  num_workers=val_num_workers,
  persistent_workers=persistent_workers,
  pin_memory=True,
  drop_last=False,
  sampler=dict(type='DefaultSampler', shuffle=False),
  dataset=dict(
      type=dataset_type,
      data_root=data_root,
      metainfo=metainfo,
      ann_file=test_ann_file,
      data_prefix=dict(img=test_data_prefix),
      test_mode=True,
      batch_shapes_cfg=batch_shapes_cfg,
      pipeline=test_pipeline)
)

# Modify classes
num_classes=20

# Modify metric related settings
val_evaluator = dict(outfile_prefix='./work_dirs/coco_rtmdet/val')
test_evaluator = dict(outfile_prefix='./work_dirs/coco_rtmdet/test')

# Use the pre-trained model
load_from = '/content/mmyolo/checkpoints/custom_tiny_rtmdet_coco_20230824.pth'
"""

with open('./configs/rtmdet/distillation/custom_tiny_rtmdet_coco.py', 'w') as f:
  f.write(config_rtmdet_tiny)

In [ ]:
# Train a model
# %cd /content/mmyolo/

!python tools/train.py configs/rtmdet/distillation/custom_tiny_rtmdet_coco.py

# 3.Inference

In [ ]:
# Unzip the mmdetection saved with downloaded checkpoint file
import zipfile

ANNO_ZIP = '/content/drive/MyDrive/AIB_tp2/model/mmdetection.zip'

# OUTPUT_DIR = '/content/data/coco'
OUTPUT_DIR = '/content/'

with zipfile.ZipFile(ANNO_ZIP, 'r') as zip_ref:
  zip_ref.extractall(OUTPUT_DIR)

print("Successfully extracted the zip file.")

!mv /content/content/mmdetection ../
%rm -rf /content/content

In [ ]:
# Verify the installation
import mmdet
print(mmdet.__version__)

In [ ]:
%cd /content/mmdetection
!pip install -e .

In [ ]:
# Download a model - Pretrained Model weights
!pwd
%cd /content/mmdetection/
!mkdir checkpoints

!wget -c https://download.openmmlab.com/mmdetection/v3.0/rtmdet/rtmdet_tiny_8xb32-300e_coco/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth\
-O checkpoints/rtmdet_tinay_coco_20230824.pth

In [ ]:
# Create a config file
%cd /content/mmdetection/
!cp configs/rtmdet/rtmdet_tiny_8xb32-300e_coco.py configs/rtmdet/custom_rtmdet_tiny_coco.py

In [ ]:
# Modify a config file
config_rtmdet="""
_base_='rtmdet_tiny_8xb32-300e_coco.py'

# Modify num_classes
model = dict(
    bbox_head=dict(num_classes=35))

# Modify dataset related settings
data_root = '/content/data/coco/'
metainfo = {
    'classes': ('인도', '횡단보도', '자전거 도로','교차로', '중앙 차선','안전지대',\
                 '정지선','정지선 위반 판별구역','보행자 신호등 녹색','보행자 신호등 적색',\
                 '차량 신호등 녹색', '차량 신호등 적색',\
                 '오토바이', '오토바이_보행자도로 통행위반', '오토바이_안전모 미착용',\
                 '오토바이_무단횡단','오토바이_신호위반','오토바이_정지선위반',\
                 '오토바이_횡단보도 주행위반','자전거','자전거 캐리어',\
                 '자전거_보행자도로 통행위반', '자전거_안전모 미착용', '자전거_무단횡단',\
                 '자전거_신호위반', '자전거_정지선위반', '자전거_횡단보도 주행위반',\
                 '킥보드', '킥보드 캐리어', '킥보드_보행자도로 통행위반',\
                 '킥보드_안전모 미착용', '킥보드_무단횡단', '킥보드_신호위반',\
                 '킥보드_횡단보도 주행위반', '킥보드_동승자 탑승위반'),
    'palette': [
        (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255),\
        (0, 255, 255), (255, 128, 0), (128, 0, 255), (0, 255, 128), (255, 128, 128),\
        (128, 255, 128), (128, 128, 255), (128, 128, 0), (128, 0, 128), (0, 128, 128),\
        (192, 64, 0), (192, 192, 64), (64, 192, 192), (64, 64, 192), (192, 64, 192),\
        (64, 192, 64), (255, 192, 128), (128, 255, 192), (128, 192, 255), (51, 102, 255),\
         (51, 204, 204), (153, 204, 0), (255, 204, 0), (255, 153, 0), (255, 102, 0),\
         (204, 255, 255), (255, 255, 153), (153, 204, 255), (255, 153, 204), (204, 153, 255),\
         (255, 204, 153), (51, 153, 102)
    ]
}
train_dataloader = dict(
    batch_size=16,
    num_workers=4,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotations/train_annotations.json',
        data_prefix=dict(img='train/')))
val_dataloader = dict(
    batch_size=16,
    num_workers=4,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotations/val_annotations.json',
        data_prefix=dict(img='val/')))
test_dataloader = dict(
    batch_size=16,
    num_workers=4,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        ann_file='annotations/test_annotations.json',
        data_prefix=dict(img='test/')))

# Modify metric related settings
val_evaluator = dict(ann_file=data_root + 'annotations/val_annotations.json',
outfile_prefix='./work_dirs/coco_rtmdet/val')
test_evaluator = dict(ann_file=data_root + 'annotations/test_annotations.json',
outfile_prefix='./work_dirs/coco_rtmdet/test')

# Modify epoch
train_cfg=dict(max_epochs=10, val_interval=5)

# Auto_LR_Scaler On
auto_scale_lr = dict(enable=True)

# Use the pre-trained model
load_from = '/content/mmdetection/checkpoints/rtmdet_tinay_coco_20230824.pth'
"""

with open('./configs/rtmdet/custom_rtmdet_tiny_coco.py', 'w') as f:
  f.write(config_rtmdet)

In [ ]:
# Train a mdoel
# %%time
!python tools/train.py configs/rtmdet/custom_rtmdet_tiny_coco.py

In [ ]:
# Save the checkpoint file (.pth) into Google Drive
%cd /content/mmdetection/work_dirs/

!zip -r /content/drive/MyDrive/AIB_tp2/model/rtmdet.zip /content/mmdetection/work_dirs